### ChatMessageHistory(基础)

In [1]:
#1.导入相关包
from langchain.memory import ChatMessageHistory

#2.实例化ChatMessageHistory对象
history = ChatMessageHistory()

# 3.添加UserMessage
history.add_user_message("hi!")

# 4.添加AIMessage
history.add_ai_message("whats up?")

# 5.返回存储的所有消息列表
history.messages

/opt/anaconda3/envs/langchain/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='whats up?', additional_kwargs={}, response_metadata={})]

In [4]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

history.add_ai_message("我是一个无所不能的小智")
history.add_user_message("你好，我叫小明，请介绍一下你自己")
history.add_user_message("我是谁呢？")

# print(history.messages)  #返回List[BaseMessage]类型
# 创建LLM
import os

load_dotenv()
llm = ChatOpenAI(
    model="qwen-plus",
    api_key=os.getenv('DASHSCOPE_API_KEY'),
    base_url=os.getenv('DASHSCOPE_BASE_URL')
)

llm.invoke(history.messages)

AIMessage(content='你好小明！我是通义千问，英文名Qwen，是一个超厉害的AI助手。我可以帮你写故事、写邮件、做计算，还能陪你聊天解闷。你可以问我任何问题，或者让我帮你完成各种任务。虽然我不能说自己是无所不能啦，但我真的很擅长很多事情哦！说说看你想让我帮你做什么呢？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 42, 'total_tokens': 117, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-f82665db-4388-9f08-b012-3154d70de5ec', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--f2fcaa01-5bef-4173-a142-9ed4f1793b2a-0', usage_metadata={'input_tokens': 42, 'output_tokens': 75, 'total_tokens': 117, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

### ConversationBufferMemory

ConversationBufferMemory是一个基础的`对话记忆（Memory）组件`，专门用于按`原始顺序存储`完整的对话历史。它的核心特点是`简单`、`无裁剪`、`无压缩`，适用于需要完整上下文的小规模对话场景。

### 使用PromptTemplate

In [25]:
# 1.导入相关包
from langchain.memory import ConversationBufferMemory

# 2.实例化ConversationBufferMemory对象
memory = ConversationBufferMemory(return_messages=False)
# 3.保存消息到内存中
memory.save_context(inputs = {"input": "你好，我是人类"}, outputs = {"output": "你好，我是AI助手"})
memory.save_context(inputs = {"input": "很开心认识你"}, outputs = {"output": "我也是"})

# 4.读取内存中消息（返回消息内容的纯文本）
print(memory.load_memory_variables({}))
print(memory.chat_memory.messages)

{'history': 'Human: 你好，我是人类\nAI: 你好，我是AI助手\nHuman: 很开心认识你\nAI: 我也是'}
[HumanMessage(content='你好，我是人类', additional_kwargs={}, response_metadata={}), AIMessage(content='你好，我是AI助手', additional_kwargs={}, response_metadata={}), HumanMessage(content='很开心认识你', additional_kwargs={}, response_metadata={}), AIMessage(content='我也是', additional_kwargs={}, response_metadata={})]


#### 注意：
- 不管inputs、outputs的key用什么名字，都认为inputs的key是human，outputs的key是AI。
- 打印的结果的json数据的key，默认是“history”。可以通过ConversationBufferMemory的`memory_key`属性修改。

In [26]:
# 1.导入相关包
from langchain.memory import ConversationBufferMemory

# 2.实例化ConversationBufferMemory对象
memory = ConversationBufferMemory(return_messages=True)

# 3.保存消息到内存中
memory.save_context({"input": "hi"}, {"output": "whats up"})

# 4.读取内存中消息（返回消息）
print(memory.load_memory_variables({}))

# 5.读取内存中消息( 访问原始消息列表)
print(memory.chat_memory.messages)

{'history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='whats up', additional_kwargs={}, response_metadata={})]}
[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='whats up', additional_kwargs={}, response_metadata={})]


#### 使用PromptTemplate结合LLM使用

In [18]:
from langchain_openai import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate

# 创建提示
# 有两个输入键：实际输入与来自记忆类的输入 需确保PromptTemplate和ConversationBufferMemory中的键匹配
template = """你可以与人类对话。

当前对话: {history}

人类问题: {question}

回复:
"""

prompt = PromptTemplate.from_template(template)

# 创建ConversationBufferMemory
memory = ConversationBufferMemory()

# 初始化链
chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

# 提问
res1 = chain.invoke({"question": "我的名字叫Tom"})
print(res1)
print(memory.load_memory_variables({}))

{'question': '我的名字叫Tom', 'history': '', 'text': '你好，Tom！有什么我可以帮你的吗？'}
{'history': 'Human: 我的名字叫Tom\nAI: 你好，Tom！有什么我可以帮你的吗？'}


In [19]:
res = chain.invoke({"question": "我的名字是什么?"})
print(res)

{'question': '我的名字是什么?', 'history': 'Human: 我的名字叫Tom\nAI: 你好，Tom！有什么我可以帮你的吗？', 'text': '你的名字是Tom！'}


In [20]:
from langchain_openai import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate


# 创建提示
# 有两个输入键：实际输入与来自记忆类的输入 需确保PromptTemplate和ConversationBufferMemory中的键匹配
template = """你可以与人类对话。

当前对话: {chat_history}

人类问题: {question}

回复:
"""

prompt = PromptTemplate.from_template(template)

# 创建ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history")

# 初始化链
chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

# 提问
res1 = chain.invoke({"question": "我的名字叫Tom"})
print(str(res1) + "\n")
print(memory.load_memory_variables({}))
res = chain.invoke({"question": "我的名字是什么?"})
print(res)

{'question': '我的名字叫Tom', 'chat_history': '', 'text': '你好，Tom！很高兴认识你。有什么我可以帮你的吗？'}

{'chat_history': 'Human: 我的名字叫Tom\nAI: 你好，Tom！很高兴认识你。有什么我可以帮你的吗？'}
{'question': '我的名字是什么?', 'chat_history': 'Human: 我的名字叫Tom\nAI: 你好，Tom！很高兴认识你。有什么我可以帮你的吗？', 'text': '你的名字是Tom！'}


#### 使用ChatPromptTemplate 和 return_messages

In [29]:
# 1.导入相关包
from langchain_core.messages import SystemMessage
from langchain.chains.llm import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import MessagesPlaceholder,ChatPromptTemplate,HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI



# 3.创建Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system","你是一个与人类对话的机器人。"),
    MessagesPlaceholder(variable_name='history'),
    ("human","问题：{question}")
])

# 4.创建Memory
memory = ConversationBufferMemory(return_messages=True)
# 5.创建LLMChain
llm_chain = LLMChain(prompt=prompt,llm=llm, memory=memory)

# 6.调用LLMChain
res1 = llm_chain.invoke({"question": "中国首都在哪里？"})
print(res1,end="\n\n")

res2 = llm_chain.invoke({"question": "我刚刚问了什么"})
print(res2)


{'question': '中国首都在哪里？', 'history': [HumanMessage(content='中国首都在哪里？', additional_kwargs={}, response_metadata={}), AIMessage(content='中国首都是**北京**。北京是中国的政治、文化和国际交往中心，也是世界著名古都之一。', additional_kwargs={}, response_metadata={})], 'text': '中国首都是**北京**。北京是中国的政治、文化和国际交往中心，也是世界著名古都之一。'}

{'question': '我刚刚问了什么', 'history': [HumanMessage(content='中国首都在哪里？', additional_kwargs={}, response_metadata={}), AIMessage(content='中国首都是**北京**。北京是中国的政治、文化和国际交往中心，也是世界著名古都之一。', additional_kwargs={}, response_metadata={}), HumanMessage(content='我刚刚问了什么', additional_kwargs={}, response_metadata={}), AIMessage(content='你刚刚问的是：“中国首都在哪里？”', additional_kwargs={}, response_metadata={})], 'text': '你刚刚问的是：“中国首都在哪里？”'}


### ConversationChain

ConversationChain提供了包含AI角色和人类角色的对话摘要格式，这个对话格式和记忆机制结合得非常紧密。

ConversationChain实际上是就是对`ConversationBufferMemory`和`LLMChain`进行了封装，并且提供一个默认格式的提示词模版（我们也可以不用），从而简化了初始化ConversationBufferMemory的步骤。

举例1：使用PromptTemplate

In [32]:
from langchain.chains.conversation.base import ConversationChain
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains import LLMChain
# 初始化大模型
llm=ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv('OPENAI_API_KEY'),
    base_url=os.getenv('OPENAI_BASE_URL')
)
template = """以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会真诚地表示不知道。

当前对话：
{history}
Human: {input}
AI:"""

prompt = PromptTemplate.from_template(template)

# memory = ConversationBufferMemory()
#
# conversation = LLMChain(
#     llm=llm,
#     prompt = prompt,
#     memory=memory,
#     verbose=True,
# )

chain = ConversationChain(llm = llm, prompt = prompt,verbose=True)


chain.invoke({"input":"你好，你的名字叫小智"})  #注意，chain中的key必须是input，否则会报错



> Entering new ConversationChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会真诚地表示不知道。

当前对话：

Human: 你好，你的名字叫小智
AI:

> Finished chain.


{'input': '你好，你的名字叫小智',
 'history': '',
 'response': '你好！是的，我叫小智，很高兴认识你！有什么我可以帮助你的吗？'}

In [33]:
chain.invoke({"input":"你好，你叫什么名字？"})



> Entering new ConversationChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会真诚地表示不知道。

当前对话：
Human: 你好，你的名字叫小智
AI: 你好！是的，我叫小智，很高兴认识你！有什么我可以帮助你的吗？
Human: 你好，你叫什么名字？
AI:

> Finished chain.


{'input': '你好，你叫什么名字？',
 'history': 'Human: 你好，你的名字叫小智\nAI: 你好！是的，我叫小智，很高兴认识你！有什么我可以帮助你的吗？',
 'response': '你好！我叫小智，很高兴再次见到你！请问有什么我可以帮助你的吗？'}

In [34]:
# 1.导入所需的库
from langchain_openai import ChatOpenAI
from langchain.chains.conversation.base import ConversationChain


# 3.初始化对话链
conv_chain = ConversationChain(llm=llm)

# 4.进行对话
resut1 = conv_chain.invoke(input="小明有1只猫")
# print(resut1)
resut2 = conv_chain.invoke(input="小刚有2只狗")
# print(resut2)
resut3 = conv_chain.invoke(input="小明和小刚一共有几只宠物?")
print(resut3)

{'input': '小明和小刚一共有几只宠物?', 'history': 'Human: 小明有1只猫\nAI: 小明的猫是什么品种呢？猫咪的颜色和性别是怎样的？另外，小明平时怎么照顾这只猫？比如，给它喂什么食物，常常带它去哪里玩？猫咪也有很多有趣的行为，比如喜欢打盹、抓东西或者和人互动，你觉得小明的猫有什么特别的地方吗？\nHuman: 小刚有2只狗\nAI: 小刚的狗狗们听起来很可爱！它们是什么品种呢？狗狗的名字是什么？以及它们的性别和年龄又是怎样的？小刚平时是怎么照顾这两只狗的？比如，给它们提供什么样的食物，定期带它们去散步、游玩或者参加训练课程吗？狗狗的性格也很有趣，有的活泼好动，有的则比较温顺。你觉得小刚的狗有什么特别之处？它们之间有没有有趣的互动呢？', 'response': '小明有1只猫，小刚有2只狗，所以他们一共有3只宠物！这真是一群让人愉快的小伙伴。你觉得小明和小刚的宠物们会不会互动呢？比如一起玩或者一起出去散步？这一定会很有趣！'}


###  ConversationBufferWindowMemory

在了解了ConversationBufferMemory记忆类后，我们知道了它能够无限的将历史对话信息填充到History中，从而给大模型提供上下文的背景。但这会`导致内存量十分大`，并且`消耗的token是非常多`的，此外，每个大模型都存在最大输入的Token限制。

我们发现，过久远的对话数据往往并不能对当前轮次的问答提供有效的信息，LangChain 给出的解决方式是：`ConversationBufferWindowMemory`模块。该记忆类会`保存一段时间内对话交互`的列表，`仅使用最近 K 个交互`。这样就使缓存区不会变得太大。

In [35]:
# 1.导入相关包
from langchain.memory import ConversationBufferWindowMemory

# 2.实例化ConversationBufferWindowMemory对象，设定窗口阈值
memory = ConversationBufferWindowMemory(k=2)
# 3.保存消息
memory.save_context({"input": "你好"}, {"output": "怎么了"})
memory.save_context({"input": "你是谁"}, {"output": "我是AI助手"})
memory.save_context({"input": "你的生日是哪天？"}, {"output": "我不清楚"})
# 4.读取内存中消息（返回消息内容的纯文本）
print(memory.load_memory_variables({}))

{'history': 'Human: 你是谁\nAI: 我是AI助手\nHuman: 你的生日是哪天？\nAI: 我不清楚'}


/var/folders/88/h2503n817r15dm_yghgn_vcm0000gn/T/ipykernel_39445/2600179964.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2)


In [36]:
# 1.导入相关包
from langchain.memory import ConversationBufferWindowMemory

# 2.实例化ConversationBufferWindowMemory对象，设定窗口阈值
memory = ConversationBufferWindowMemory(k=2, return_messages=True)
# 3.保存消息
memory.save_context({"input": "你好"}, {"output": "怎么了"})
memory.save_context({"input": "你是谁"}, {"output": "我是AI助手小智"})
memory.save_context({"input": "初次对话，你能介绍一下你自己吗？"}, {"output": "当然可以了。我是一个无所不能的小智。"})
# 4.读取内存中消息（返回消息内容的纯文本）
print(memory.load_memory_variables({}))

{'history': [HumanMessage(content='你是谁', additional_kwargs={}, response_metadata={}), AIMessage(content='我是AI助手小智', additional_kwargs={}, response_metadata={}), HumanMessage(content='初次对话，你能介绍一下你自己吗？', additional_kwargs={}, response_metadata={}), AIMessage(content='当然可以了。我是一个无所不能的小智。', additional_kwargs={}, response_metadata={})]}


In [37]:
from langchain.memory import ConversationBufferWindowMemory
# 1.导入相关包
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain

# 2.定义模版
template = """以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会表示不知道。

当前对话：
{history}
Human: {question}
AI:"""

# 3.定义提示词模版
prompt_template = PromptTemplate.from_template(template)


# 5.实例化ConversationBufferWindowMemory对象，设定窗口阈值
memory = ConversationBufferWindowMemory(k=1)

# 6.定义LLMChain
conversation_with_summary = LLMChain(
    llm=llm,
    prompt=prompt_template,
    memory=memory,
    verbose=True,
)

# 7.执行链（第一次提问）
respon1 = conversation_with_summary.invoke({"question":"你好，我是孙小空"})
# print(respon1)
# 8.执行链（第二次提问）
respon2 =conversation_with_summary.invoke({"question":"我还有两个师弟，一个是猪小戒，一个是沙小僧"})
# print(respon2)
# 9.执行链（第三次提问）
respon3 =conversation_with_summary.invoke({"question":"我今年高考，竟然考上了1本"})
# print(respon3)
# 10.执行链（第四次提问）
respon4 =conversation_with_summary.invoke({"question":"我叫什么？"})
print(respon4)



> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会表示不知道。

当前对话：

Human: 你好，我是孙小空
AI:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会表示不知道。

当前对话：
Human: 你好，我是孙小空
AI: 你好，孙小空！很高兴认识你！你今天怎么样？有什么想聊的或者想了解的话题吗？
Human: 我还有两个师弟，一个是猪小戒，一个是沙小僧
AI:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会表示不知道。

当前对话：
Human: 我还有两个师弟，一个是猪小戒，一个是沙小僧
AI: 哇，听起来你们的名字都很有趣！猪小戒和沙小僧有什么特别的故事或者特点吗？你们之间有些什么样的互动呢？
Human: 我今年高考，竟然考上了1本
AI:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会表示不知道。

当前对话：
Human: 我今年高考，竟然考上了1本
AI: 太棒了！恭喜你考上了本科！这是一个重要的里程碑，你一定付出了很多努力。你打算学什么专业呢？或者有什么未来的计划吗？
Human: 我叫什么？
AI:

> Finished chain.
{'question': '我叫什么？', 'history': 'Human: 我今年高考，竟然考上了1本\nAI: 太棒了！恭喜你